In [ ]:
#%% En el caso de Google Colaboratory, es necesario instalar las librerías necesarias para todos los análisis que se muestran adelante

In [ ]:
pip install dipy

In [ ]:
pip install nilearn

In [ ]:
#%% Conectar con google drive, donde se encuentran los datos HCP1200 requeridos
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
#%% Se importa el archivo con las funciones definidas para la tractografia y extraer la conectividad estructural

execfile('/gdrive/Shareddrives/GENSyR /CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/Tesis Juan y Diego /Resultados FUNCIONES Tesis/Construcción redes cerebrales/construcción_conectividad_estructural.py')

In [ ]:
#%% Montar librerías base para todo el proceso

import os
import sys
import subprocess
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from os.path import expanduser, join


#%% Importar parcelación que se usará para definir los nodos de la red y ruta para guardar la matriz.
from dipy.io.image import load_nifti_data, load_nifti, save_nifti

In [ ]:
#%% Leer y cargar las rutas de los datos del HCP
home = expanduser('/gdrive/Shareddrives/GENSyR /CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/Datos Proyecto Conectoma Humano (HCP)')

In [ ]:
fpardic = join(home, 'Parcelas', 'dict_pars.npy')
arr_dict_pars = np.load(fpardic, allow_pickle=True)
dict_pars = arr_dict_pars.item()

In [ ]:
dict_pars

In [ ]:
pars_keys = list(dict_pars.keys())
index = pars_keys.index('AAL')
par_key = [pars_keys[index]]

In [ ]:
# Ruta donde se tienen las listas de sujetos, separadas por poblaciones para analizar
fdatapoblaciones = '/gdrive/Shareddrives/GENSyR /CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/Tesis Juan y Diego /Datos Poblaciones/Sujetos Tesis'

# Se tienen los filtros de tareas 'WM_Task_Acc' o 'Dexterity_Unadj'
tareas = ['Emotion', 'Social', 'WM']
# Para las poblaciones de sujetos masculinas 'M' y femeninas 'F'
sexos = ['F', 'M']
# Puntuacion de la poblacion: alto '+' o bajo '-'
puntuaciones = ['-', '+']

In [ ]:
for tarea in tareas:
  for sexo in sexos:
    for puntuacion in puntuaciones:

      # Se define la carpeta donde estan los sujetos con la poblacion de interes
      patient = tarea + '_' + sexo + puntuacion

      # Se cargan las rutas para leer el tractograma
      dhcp = join(fdatapoblaciones, patient)
      dest = join(dhcp, 'Estructural')

      # Se genera la ruta para leer el tractograma
      nametracto = 'trac_det_EuDX_pft_' + patient + '.trk'
      ftrackpft = join(dest, nametracto)

      # Cargar el archivo trk en streamlines
      streamsObjIN=nib.streamlines.load(ftrackpft)
      streamlines = streamsObjIN.tractogram.streamlines

      for parcelacion in par_key: # pars_keys

        fparcelas = dict_pars[parcelacion]
        par_data, par_affine= load_nifti(fparcelas)

        # Se verifica si la carpeta de parcelas para cada sujeto existe
        #dparcelas = join(dhcp, 'Parcelas')
        #if not os.path.exists(dparcelas):
        #  os.mkdir(dparcelas)

        # Se verifica si la carpeta para guardar los datos de la matriz estructural para una parcela especifica existe
        #dpar = join(dparcelas, parcelacion)
        #if not os.path.exists(dpar):
        #  os.mkdir(dpar)

        # Se ejecuta la funcion para crear las matrices estructurales
        conectividades_estructurales(patient, streamlines, '', par_data, par_affine, dhcp)